In [7]:
#importiere libraries
library(reshape2)
library(ggplot2)
library(ggrepel)
library(corrplot)
library(gridExtra)
#library(ggpubr)

#lade csv file als data.frame rawData
rawData = read.csv("QuantitativeDaten.csv", header = TRUE, stringsAsFactors = FALSE)

#erstelle neue Spalte mit Produkttyp ("Plastik" vs "Alternativ")
rawData$Typ = "Plastik"
rawData[startsWith(rawData$Produkt, "A"), "Typ"] = "Alternativ"

#erstelle neue Spalte mit Produktkategorie
rawData$Kategorie = substr(rawData$Produkt, 2, 2)
rawData$Kategorie = as.integer(rawData$Kategorie)

#daten anzeigen
head(rawData)

#Zusammenfassung anzeigen
summary(rawData)

VP,VB,Geschlecht,Produkt,B3_Fragen,B1_Präf,B1_Fragen,B2_Kennen,B2_Besitzen,B2_Kaufen,B2_Ästhetik,B2_Alltagstaugl.,B2_Kosten,B2_Kosten_Klingen,B3_Präf,Präf_Diff,Umwelt.FB,Typ,Kategorie
<int>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,<chr>,<int>
1,Ökonomisch,w,P6,0,0,0,1,0,0,2,1,2,0,0,0,119,Plastik,6
1,Ökonomisch,w,A6,0,0,0,1,0,0,4,1,12,0,0,0,119,Alternativ,6
1,Ökonomisch,w,A6.5,0,0,0,1,0,0,2,1,12,0,0,0,119,Alternativ,6
1,Ökonomisch,w,P5,0,0,0,1,1,1,4,6,2,0,0,0,119,Plastik,5
1,Ökonomisch,w,A5,0,2,1,1,1,1,4,6,5,0,0,-2,119,Alternativ,5
1,Ökonomisch,w,P7,0,0,0,1,1,1,4,5,2,0,0,0,119,Plastik,7


       VP             VB             Geschlecht          Produkt         
 Min.   : 1.00   Length:768         Length:768         Length:768        
 1st Qu.:12.75   Class :character   Class :character   Class :character  
 Median :24.50   Mode  :character   Mode  :character   Mode  :character  
 Mean   :24.50                                                           
 3rd Qu.:36.25                                                           
 Max.   :48.00                                                           
   B3_Fragen         B1_Präf        B1_Fragen        B2_Kennen     
 Min.   :0.0000   Min.   :0.000   Min.   :0.0000   Min.   :0.0000  
 1st Qu.:0.0000   1st Qu.:0.000   1st Qu.:0.0000   1st Qu.:1.0000  
 Median :0.0000   Median :0.000   Median :0.0000   Median :1.0000  
 Mean   :0.2656   Mean   :1.169   Mean   :0.3398   Mean   :0.8672  
 3rd Qu.:1.0000   3rd Qu.:3.000   3rd Qu.:1.0000   3rd Qu.:1.0000  
 Max.   :2.0000   Max.   :3.000   Max.   :2.0000   Max.   :1.0000  
  B2_B

In [8]:
#Datenformat normieren
data = melt(rawData, id.var = c("VP", "Geschlecht", "VB", "Produkt", "Kategorie", "Typ"), variable.name = "Frage", value.name = "Wert")
data$Frage = as.character(data$Frage)

#Daten anzeigen
head(data)

#usammenfassung anzeigen
summary(data)

VP,Geschlecht,VB,Produkt,Kategorie,Typ,Frage,Wert
<int>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<dbl>
1,w,Ökonomisch,P6,6,Plastik,B3_Fragen,0
1,w,Ökonomisch,A6,6,Alternativ,B3_Fragen,0
1,w,Ökonomisch,A6.5,6,Alternativ,B3_Fragen,0
1,w,Ökonomisch,P5,5,Plastik,B3_Fragen,0
1,w,Ökonomisch,A5,5,Alternativ,B3_Fragen,0
1,w,Ökonomisch,P7,7,Plastik,B3_Fragen,0


       VP         Geschlecht             VB              Produkt         
 Min.   : 1.00   Length:9984        Length:9984        Length:9984       
 1st Qu.:12.75   Class :character   Class :character   Class :character  
 Median :24.50   Mode  :character   Mode  :character   Mode  :character  
 Mean   :24.50                                                           
 3rd Qu.:36.25                                                           
 Max.   :48.00                                                           
   Kategorie         Typ               Frage                Wert        
 Min.   :1.000   Length:9984        Length:9984        Min.   : -3.000  
 1st Qu.:2.750   Class :character   Class :character   1st Qu.:  0.000  
 Median :4.000   Mode  :character   Mode  :character   Median :  1.000  
 Mean   :4.062                                         Mean   :  9.899  
 3rd Qu.:6.000                                         3rd Qu.:  3.000  
 Max.   :7.000                              

In [11]:
#Boxplot-Funktion für ein Feature
drawAvgDataFeatureByProductBoxplots = function(feature){
  # Datenset selektieren
  dataFeature = data[data$Frage == feature,]
  # Durchschnitt pro Frage berechnen
  avgDataFeatureByProduct = with(dataFeature, aggregate(Wert ~ Produkt + Typ, FUN = mean))
  # Boxplots für avgFeatDataByProduct erstellen
  avgDataFeatureByProductBoxplots = ggplot(avgDataFeatureByProduct, aes(x = Typ, y = Wert, label = Produkt)) +
    geom_boxplot() +
    geom_point() +
    geom_label_repel()  
  
  # Print Boxplots
  print(avgDataFeatureByProduct)
  print(avgDataFeatureByProductBoxplots)
  
  # Print Quantile
  print(summary(avgDataFeatureByProduct[avgDataFeatureByProduct$Typ == 'Alternativ',]))
  print(summary(avgDataFeatureByProduct[avgDataFeatureByProduct$Typ == 'Plastik',]))
  
  # Print Standartabweichung
  print(sd(avgDataFeatureByProduct[avgDataFeatureByProduct$Typ == 'Alternativ',]$Wert))
  print(sd(avgDataFeatureByProduct[avgDataFeatureByProduct$Typ == 'Plastik',]$Wert))
  
  }

In [15]:
#erstelle boxplot für B1_Präf
drawAvgDataFeatureByProductBoxplots("B1_Präf")

ggsave("B1_Präf.png")

   Produkt        Typ      Wert
1       A1 Alternativ 2.3541667
2       A2 Alternativ 2.2708333
3       A3 Alternativ 1.8958333
4     A3.5 Alternativ 1.8958333
5       A4 Alternativ 0.2916667
6       A5 Alternativ 0.6875000
7       A6 Alternativ 1.7708333
8     A6.5 Alternativ 1.7708333
9       A7 Alternativ 0.6458333
10      P1    Plastik 0.1666667
11      P2    Plastik 0.1250000
12      P3    Plastik 0.2916667
13      P4    Plastik 1.8333333
14      P5    Plastik 1.2500000
15      P6    Plastik 0.3125000
16      P7    Plastik 1.1458333
   Produkt              Typ                 Wert       
 Length:9           Length:9           Min.   :0.2917  
 Class :character   Class :character   1st Qu.:0.6875  
 Mode  :character   Mode  :character   Median :1.7708  
                                       Mean   :1.5093  
                                       3rd Qu.:1.8958  
                                       Max.   :2.3542  
   Produkt              Typ                 Wert       
 Length:

Saving 6.67 x 6.67 in image


In [16]:
#erstelle boxplot für B1_Fragen
drawAvgDataFeatureByProductBoxplots("B1_Fragen")

ggsave("B1_Fragen.png")

   Produkt        Typ      Wert
1       A1 Alternativ 0.5625000
2       A2 Alternativ 0.3333333
3       A3 Alternativ 0.4583333
4     A3.5 Alternativ 0.4583333
5       A4 Alternativ 0.5625000
6       A5 Alternativ 0.6250000
7       A6 Alternativ 0.3750000
8     A6.5 Alternativ 0.3750000
9       A7 Alternativ 0.6041667
10      P1    Plastik 0.1041667
11      P2    Plastik 0.1041667
12      P3    Plastik 0.2500000
13      P4    Plastik 0.1458333
14      P5    Plastik 0.2708333
15      P6    Plastik 0.0000000
16      P7    Plastik 0.2083333
   Produkt              Typ                 Wert       
 Length:9           Length:9           Min.   :0.3333  
 Class :character   Class :character   1st Qu.:0.3750  
 Mode  :character   Mode  :character   Median :0.4583  
                                       Mean   :0.4838  
                                       3rd Qu.:0.5625  
                                       Max.   :0.6250  
   Produkt              Typ                 Wert       
 Length:

Saving 6.67 x 6.67 in image


In [17]:
#erstelle boxplot für B2_Kennen
drawAvgDataFeatureByProductBoxplots("B2_Kennen")

ggsave("B2_Kennen.png")

   Produkt        Typ      Wert
1       A1 Alternativ 0.9583333
2       A2 Alternativ 0.6041667
3       A3 Alternativ 0.9583333
4     A3.5 Alternativ 0.9375000
5       A4 Alternativ 0.4166667
6       A5 Alternativ 0.7708333
7       A6 Alternativ 0.7291667
8     A6.5 Alternativ 0.7291667
9       A7 Alternativ 0.7708333
10      P1    Plastik 1.0000000
11      P2    Plastik 1.0000000
12      P3    Plastik 1.0000000
13      P4    Plastik 1.0000000
14      P5    Plastik 1.0000000
15      P6    Plastik 1.0000000
16      P7    Plastik 1.0000000
   Produkt              Typ                 Wert       
 Length:9           Length:9           Min.   :0.4167  
 Class :character   Class :character   1st Qu.:0.7292  
 Mode  :character   Mode  :character   Median :0.7708  
                                       Mean   :0.7639  
                                       3rd Qu.:0.9375  
                                       Max.   :0.9583  
   Produkt              Typ                 Wert  
 Length:7    

Saving 6.67 x 6.67 in image


In [18]:
#erstelle boxplot für B2_Besitzen
drawAvgDataFeatureByProductBoxplots("B2_Besitzen")
ggsave("B2_Besitzen.png")

   Produkt        Typ       Wert
1       A1 Alternativ 0.60416667
2       A2 Alternativ 0.22916667
3       A3 Alternativ 0.37500000
4     A3.5 Alternativ 0.43750000
5       A4 Alternativ 0.02083333
6       A5 Alternativ 0.31250000
7       A6 Alternativ 0.10416667
8     A6.5 Alternativ 0.10416667
9       A7 Alternativ 0.20833333
10      P1    Plastik 0.37500000
11      P2    Plastik 0.45833333
12      P3    Plastik 0.41666667
13      P4    Plastik 0.85416667
14      P5    Plastik 0.87500000
15      P6    Plastik 0.27083333
16      P7    Plastik 0.87500000
   Produkt              Typ                 Wert        
 Length:9           Length:9           Min.   :0.02083  
 Class :character   Class :character   1st Qu.:0.10417  
 Mode  :character   Mode  :character   Median :0.22917  
                                       Mean   :0.26620  
                                       3rd Qu.:0.37500  
                                       Max.   :0.60417  
   Produkt              Typ             

Saving 6.67 x 6.67 in image


In [19]:
#erstelle boxplot für B2_Kaufen
drawAvgDataFeatureByProductBoxplots("B2_Kaufen")

ggsave("B2_Kaufen.png")

   Produkt        Typ      Wert
1       A1 Alternativ 0.5416667
2       A2 Alternativ 0.6041667
3       A3 Alternativ 0.7291667
4     A3.5 Alternativ 0.6458333
5       A4 Alternativ 0.4791667
6       A5 Alternativ 0.7291667
7       A6 Alternativ 0.3333333
8     A6.5 Alternativ 0.3333333
9       A7 Alternativ 0.5833333
10      P1    Plastik 0.4166667
11      P2    Plastik 0.4166667
12      P3    Plastik 0.2083333
13      P4    Plastik 0.7500000
14      P5    Plastik 0.8125000
15      P6    Plastik 0.1250000
16      P7    Plastik 0.7708333
   Produkt              Typ                 Wert       
 Length:9           Length:9           Min.   :0.3333  
 Class :character   Class :character   1st Qu.:0.4792  
 Mode  :character   Mode  :character   Median :0.5833  
                                       Mean   :0.5532  
                                       3rd Qu.:0.6458  
                                       Max.   :0.7292  
   Produkt              Typ                 Wert       
 Length:

Saving 6.67 x 6.67 in image


In [20]:
#erstelle boxplot für B2_Ästhetik
drawAvgDataFeatureByProductBoxplots("B2_Ästhetik")

ggsave("B2_Ästhetik.png")

   Produkt        Typ     Wert
1       A1 Alternativ 5.437500
2       A2 Alternativ 4.687500
3       A3 Alternativ 5.937500
4     A3.5 Alternativ 5.666667
5       A4 Alternativ 5.208333
6       A5 Alternativ 4.562500
7       A6 Alternativ 4.916667
8     A6.5 Alternativ 4.291667
9       A7 Alternativ 4.770833
10      P1    Plastik 2.541667
11      P2    Plastik 1.291667
12      P3    Plastik 4.354167
13      P4    Plastik 4.062500
14      P5    Plastik 4.000000
15      P6    Plastik 2.687500
16      P7    Plastik 4.145833
   Produkt              Typ                 Wert      
 Length:9           Length:9           Min.   :4.292  
 Class :character   Class :character   1st Qu.:4.688  
 Mode  :character   Mode  :character   Median :4.917  
                                       Mean   :5.053  
                                       3rd Qu.:5.438  
                                       Max.   :5.938  
   Produkt              Typ                 Wert      
 Length:7           Length:7     

Saving 6.67 x 6.67 in image


In [21]:
#erstelle boxplot für B2_Alltagstaugl.
drawAvgDataFeatureByProductBoxplots("B2_Alltagstaugl.")

ggsave("B2_Alltagstaugl.png")

   Produkt        Typ     Wert
1       A1 Alternativ 4.854167
2       A2 Alternativ 5.125000
3       A3 Alternativ 5.583333
4     A3.5 Alternativ 5.875000
5       A4 Alternativ 5.145833
6       A5 Alternativ 5.020833
7       A6 Alternativ 3.916667
8     A6.5 Alternativ 3.875000
9       A7 Alternativ 5.812500
10      P1    Plastik 4.291667
11      P2    Plastik 3.604167
12      P3    Plastik 5.520833
13      P4    Plastik 5.708333
14      P5    Plastik 6.125000
15      P6    Plastik 3.895833
16      P7    Plastik 6.458333
   Produkt              Typ                 Wert      
 Length:9           Length:9           Min.   :3.875  
 Class :character   Class :character   1st Qu.:4.854  
 Mode  :character   Mode  :character   Median :5.125  
                                       Mean   :5.023  
                                       3rd Qu.:5.583  
                                       Max.   :5.875  
   Produkt              Typ                 Wert      
 Length:7           Length:7     

Saving 6.67 x 6.67 in image


In [22]:
#Funktion für line chart für ein feature
drawAvgDataFeatureByProductLineChart = function(feature){
  # Datenset selektieren
  dataFeature = data[data$Frage == feature,]
  # Durchschnitt pro Frage berechnen
  avgDataFeatureByProduct = with(dataFeature, aggregate(Wert ~ Produkt + Kategorie + Typ, FUN = mean))
  # Line Charts für avgDataFeatureByProduct erstellen
  avgDataFeatureByProductLineChart = ggplot(avgDataFeatureByProduct, aes(x = Typ, y = Wert, label = Produkt, group = Kategorie)) + 
    geom_line() +
    geom_point() +
    geom_label_repel()
  
  print(avgDataFeatureByProduct)
  print(avgDataFeatureByProductLineChart)
}

In [23]:
#erstelle line chart für B2_Kosten
drawAvgDataFeatureByProductLineChart("B2_Kosten")

ggsave("B2_Kosten.png")

   Produkt Kategorie        Typ      Wert
1       A1         1 Alternativ 11.081250
2       A2         2 Alternativ  2.595625
3       A3         3 Alternativ 17.677083
4     A3.5         3 Alternativ 17.677083
5       A4         4 Alternativ 19.966458
6       A5         5 Alternativ  5.353125
7       A6         6 Alternativ  9.353750
8     A6.5         6 Alternativ  9.353750
9       A7         7 Alternativ  4.458333
10      P1         1    Plastik  0.187500
11      P2         2    Plastik  0.000000
12      P3         3    Plastik 10.946875
13      P4         4    Plastik 10.259167
14      P5         5    Plastik  2.370625
15      P6         6    Plastik  2.236667
16      P7         7    Plastik  1.869375


Saving 6.67 x 6.67 in image


In [24]:
#Vergleichstabelle für B2_Kosten_Klingen
with(data[data$Frage == "B2_Kosten_Klingen" & data$Kategorie == 4,], aggregate(Wert ~ Typ, FUN = mean))

Typ,Wert
<chr>,<dbl>
Alternativ,6.518542
Plastik,10.051875


In [25]:
#erstelle boxplot für B3_Präf
drawAvgDataFeatureByProductBoxplots("B3_Präf")

ggsave("B3_Präf.png")

   Produkt        Typ       Wert
1       A1 Alternativ 2.06250000
2       A2 Alternativ 2.27083333
3       A3 Alternativ 2.12500000
4     A3.5 Alternativ 2.12500000
5       A4 Alternativ 0.68750000
6       A5 Alternativ 1.08333333
7       A6 Alternativ 1.91666667
8     A6.5 Alternativ 1.91666667
9       A7 Alternativ 1.08333333
10      P1    Plastik 0.14583333
11      P2    Plastik 0.06250000
12      P3    Plastik 0.08333333
13      P4    Plastik 1.02083333
14      P5    Plastik 0.68750000
15      P6    Plastik 0.22916667
16      P7    Plastik 0.85416667
   Produkt              Typ                 Wert       
 Length:9           Length:9           Min.   :0.6875  
 Class :character   Class :character   1st Qu.:1.0833  
 Mode  :character   Mode  :character   Median :1.9167  
                                       Mean   :1.6968  
                                       3rd Qu.:2.1250  
                                       Max.   :2.2708  
   Produkt              Typ                 Wer

Saving 6.67 x 6.67 in image


In [26]:
#erstelle boxplot für Bräf_Diff
drawAvgDataFeatureByProductBoxplots("Präf_Diff")

ggsave("Präf_Diff.png")
```
```{r}
#erstelle boxplot für B3_Fragen
drawAvgDataFeatureByProductBoxplots("B3_Fragen")

ggsave("B3_Fragen.png")

ERROR: Error in parse(text = x, srcfile = src): attempt to use zero-length variable name


In [ ]:
#erstelle boxplot für Umwelt
envData = data[data$Frage == "Umwelt",]
ggplot(envData, aes(x = Frage, y = Wert)) +
  geom_boxplot()

envQuantiles = quantile(envData$Wert)
envQuantiles

ggsave("Umwelt.png")

In [27]:
#Funktion für gruppierte boxplots für Vergleich von P1 und P3
drawAvgDataPreferenceConditionBoxplots = function(condition){
  # selektiere Daten
  dataPreferenceCondition = data[(data$Frage == "B1_Präf" | data$Frage == "B3_Präf") & data$VB == 
                       condition,]
  # Durchschnitt pro Produkt berechnen
  avgDataPreferenceCondition = with(dataPreferenceCondition, aggregate(Wert ~ Produkt + Typ + Frage, FUN = mean))
  
  # Boxplot für avgFeatDataByProduct erstellen
  dodgeBy = position_dodge(width=0.75)
  avgDataPreferenceConditionBoxplots = ggplot(avgDataPreferenceCondition, aes(x = Typ, y = Wert, label = Produkt,      color = Frage)) +
    geom_boxplot() +
    geom_point(position = dodgeBy) +
    geom_label_repel(position = dodgeBy) +
    scale_color_manual(values=c("#9e0909", "#20458a"))
  
   # Print Quantile
  print(summary(avgDataPreferenceCondition[avgDataPreferenceCondition$Typ == 'Alternativ' & avgDataPreferenceCondition$Frage == 'B1_Präf',]))
  print(summary(avgDataPreferenceCondition[avgDataPreferenceCondition$Typ == 'Plastik' & avgDataPreferenceCondition$Frage == 'B1_Präf',]))
  print(summary(avgDataPreferenceCondition[avgDataPreferenceCondition$Typ == 'Alternativ' & avgDataPreferenceCondition$Frage == 'B3_Präf',]))
  print(summary(avgDataPreferenceCondition[avgDataPreferenceCondition$Typ == 'Plastik' & avgDataPreferenceCondition$Frage == 'B3_Präf',]))
  
  # Print Standartabweichung
  print(sd(avgDataPreferenceCondition[avgDataPreferenceCondition$Typ == 'Alternativ' & avgDataPreferenceCondition$Frage == 'B1_Präf',]$Wert))
  print(sd(avgDataPreferenceCondition[avgDataPreferenceCondition$Typ == 'Plastik' & avgDataPreferenceCondition$Frage == 'B1_Präf',]$Wert))
  print(sd(avgDataPreferenceCondition[avgDataPreferenceCondition$Typ == 'Alternativ' & avgDataPreferenceCondition$Frage == 'B3_Präf',]$Wert))
  print(sd(avgDataPreferenceCondition[avgDataPreferenceCondition$Typ == 'Plastik' & avgDataPreferenceCondition$Frage == 'B3_Präf',]$Wert))
  
  print(berechneTTestVonTypNachBlock(avgDataPreferenceCondition[avgDataPreferenceCondition$Typ == 'Alternativ',]))
  print(berechneTTestVonTypNachBlock(avgDataPreferenceCondition[avgDataPreferenceCondition$Typ == 'Plastik',]))
  
  print(avgDataPreferenceConditionBoxplots)
}

In [28]:
berechneTTestVonTypNachBlock <- function(daten)
{
  df_Produkt = unique(daten$Produkt)
  df_Wert_B1 = daten[daten$Frage == "B1_Präf","Wert"]
  df_Wert_B3 = daten[daten$Frage == "B3_Präf","Wert"]
 
  df = data.frame('Produkt' = df_Produkt, 'Wert_B1' = df_Wert_B1, 'Wert_B3' = df_Wert_B3)
  print(df)
 
  print(t.test(df$Wert_B1, df$Wert_B3))
}

In [29]:
#erstelle boxplot für Ökologisch
drawAvgDataPreferenceConditionBoxplots("Ökologisch")

ggsave("Präf_Vergleich_Ökolog.png")

   Produkt              Typ               Frage                Wert       
 Length:9           Length:9           Length:9           Min.   :0.2083  
 Class :character   Class :character   Class :character   1st Qu.:0.9167  
 Mode  :character   Mode  :character   Mode  :character   Median :1.7083  
                                                          Mean   :1.5648  
                                                          3rd Qu.:2.1250  
                                                          Max.   :2.4167  
   Produkt              Typ               Frage                Wert       
 Length:7           Length:7           Length:7           Min.   :0.1250  
 Class :character   Class :character   Class :character   1st Qu.:0.1667  
 Mode  :character   Mode  :character   Mode  :character   Median :0.2083  
                                                          Mean   :0.6607  
                                                          3rd Qu.:1.0625  
                         

Saving 6.67 x 6.67 in image


In [30]:
#Funktion für line chart für den Vergleich von P1 und P3 in je einer VB
drawAvgDataPreferenceConditionTypeLineChart = function(condition, type){
  # selektiere Datenset
  dataPreferenceConditionType = data[(data$Frage == "B1_Präf" | data$Frage == "B3_Präf") & data$VB == 
                       condition & data$Typ == type,]
  # Berechne Durchschnitt pro Produkt
  avgDataPreferenceConditionType = with(dataPreferenceConditionType, aggregate(Wert ~ Produkt + Kategorie + Frage, FUN = mean))
  # erstelle Boxplots für avgFeatDataByProduct
  avgDataPreferenceConditionTypeLineChart = ggplot(avgDataPreferenceConditionType, aes(x = Frage, y = Wert, label = Produkt, group = Kategorie)) + 
    geom_line() +
    geom_point() +
    geom_label_repel()
  
  print(avgDataPreferenceConditionType)
  print(avgDataPreferenceConditionTypeLineChart)
}

In [31]:
#erstelle line chart für VB Ökologisch - Typ Alternativ
drawAvgDataPreferenceConditionTypeLineChart("Ökologisch", "Alternativ")

ggsave("Präf_Ökolog_Alt.png")

   Produkt Kategorie   Frage      Wert
1       A1         1 B1_Präf 2.4166667
2       A2         2 B1_Präf 2.3333333
3       A3         3 B1_Präf 2.1250000
4     A3.5         3 B1_Präf 2.1250000
5       A4         4 B1_Präf 0.2083333
6       A5         5 B1_Präf 0.9166667
7       A6         6 B1_Präf 1.7083333
8     A6.5         6 B1_Präf 1.7083333
9       A7         7 B1_Präf 0.5416667
10      A1         1 B3_Präf 2.2916667
11      A2         2 B3_Präf 2.5000000
12      A3         3 B3_Präf 2.3333333
13    A3.5         3 B3_Präf 2.3333333
14      A4         4 B3_Präf 0.6250000
15      A5         5 B3_Präf 1.2916667
16      A6         6 B3_Präf 2.2083333
17    A6.5         6 B3_Präf 2.2083333
18      A7         7 B3_Präf 1.3750000


Saving 6.67 x 6.67 in image


In [32]:
#erstelle line chart für VB Ökologisch - Typ Plastik
drawAvgDataPreferenceConditionTypeLineChart("Ökologisch", "Plastik")

ggsave("Präf_Ökolog_Plast.png")

   Produkt Kategorie   Frage       Wert
1       P1         1 B1_Präf 0.12500000
2       P2         2 B1_Präf 0.12500000
3       P3         3 B1_Präf 0.20833333
4       P4         4 B1_Präf 1.83333333
5       P5         5 B1_Präf 1.04166667
6       P6         6 B1_Präf 0.20833333
7       P7         7 B1_Präf 1.08333333
8       P1         1 B3_Präf 0.12500000
9       P2         2 B3_Präf 0.00000000
10      P3         3 B3_Präf 0.04166667
11      P4         4 B3_Präf 0.87500000
12      P5         5 B3_Präf 0.54166667
13      P6         6 B3_Präf 0.12500000
14      P7         7 B3_Präf 0.66666667


Saving 6.67 x 6.67 in image


In [33]:
#erstelle gruppierte boxplots für Ökonomisch
drawAvgDataPreferenceConditionBoxplots("Ökonomisch")

ggsave("Präf_Vergleich_Ökonom.png")

   Produkt              Typ               Frage                Wert      
 Length:9           Length:9           Length:9           Min.   :0.375  
 Class :character   Class :character   Class :character   1st Qu.:0.750  
 Mode  :character   Mode  :character   Mode  :character   Median :1.667  
                                                          Mean   :1.454  
                                                          3rd Qu.:1.833  
                                                          Max.   :2.292  
   Produkt              Typ               Frage                Wert       
 Length:7           Length:7           Length:7           Min.   :0.1250  
 Class :character   Class :character   Class :character   1st Qu.:0.2917  
 Mode  :character   Mode  :character   Mode  :character   Median :0.4167  
                                                          Mean   :0.8036  
                                                          3rd Qu.:1.3333  
                                

Saving 6.67 x 6.67 in image


In [34]:
#erstelle line chart für VB Ökonomisch - Typ Alternativ
drawAvgDataPreferenceConditionTypeLineChart("Ökonomisch", "Alternativ")

ggsave("Präf_Ökonom_Alt.png")

   Produkt Kategorie   Frage      Wert
1       A1         1 B1_Präf 2.2916667
2       A2         2 B1_Präf 2.2083333
3       A3         3 B1_Präf 1.6666667
4     A3.5         3 B1_Präf 1.6666667
5       A4         4 B1_Präf 0.3750000
6       A5         5 B1_Präf 0.4583333
7       A6         6 B1_Präf 1.8333333
8     A6.5         6 B1_Präf 1.8333333
9       A7         7 B1_Präf 0.7500000
10      A1         1 B3_Präf 1.8333333
11      A2         2 B3_Präf 2.0416667
12      A3         3 B3_Präf 1.9166667
13    A3.5         3 B3_Präf 1.9166667
14      A4         4 B3_Präf 0.7500000
15      A5         5 B3_Präf 0.8750000
16      A6         6 B3_Präf 1.6250000
17    A6.5         6 B3_Präf 1.6250000
18      A7         7 B3_Präf 0.7916667


Saving 6.67 x 6.67 in image


In [35]:
#erstelle line chart für VB Ökonomisch - Typ Plastik
drawAvgDataPreferenceConditionTypeLineChart("Ökonomisch", "Plastik")

ggsave("Präf_Ökonom_Plast.png")

   Produkt Kategorie   Frage      Wert
1       P1         1 B1_Präf 0.2083333
2       P2         2 B1_Präf 0.1250000
3       P3         3 B1_Präf 0.3750000
4       P4         4 B1_Präf 1.8333333
5       P5         5 B1_Präf 1.4583333
6       P6         6 B1_Präf 0.4166667
7       P7         7 B1_Präf 1.2083333
8       P1         1 B3_Präf 0.1666667
9       P2         2 B3_Präf 0.1250000
10      P3         3 B3_Präf 0.1250000
11      P4         4 B3_Präf 1.1666667
12      P5         5 B3_Präf 0.8333333
13      P6         6 B3_Präf 0.3333333
14      P7         7 B3_Präf 1.0416667


Saving 6.67 x 6.67 in image


In [36]:
#Erstelle subset für Korrelations-Berechnung
dataType = rawData[rawData$Typ == "Alternativ", c(3,5:13,15,17)]
#dataType$VB = as.integer(as.factor(dataType$VB))
dataType$Geschlecht = as.integer(as.factor(dataType$Geschlecht))

# korrelationsn berechnen
corrMatrix = cor(dataType)

print(corrMatrix)

#CorrMatrix erstellen
cor1 = corrplot(corrMatrix, method="circle", type = "upper", sig.level = .05)
cor2 = corrplot(corrMatrix, method = "number", type = "upper", sig.level = .05, number.cex = 8/ncol(corrMatrix))

pdf(file = "cor1.pdf")
corrplot(corrMatrix, method="circle", type = "upper", sig.level = .05)
dev.off()

pdf(file = "cor2.pdf")
corrplot(corrMatrix, method = "number", type = "upper", sig.level = .05, number.cex = 8/ncol(corrMatrix))
dev.off()


#Erstelle Tabelle nur mit hoch korrelierten features
corrMatrixFlat = as.data.frame(as.table(corrMatrix), stringsAsFactors = FALSE)
colnames(corrMatrixFlat) = c("Wert1", "Wert2", "Korrelation")
corrMatrixFlat = corrMatrixFlat[corrMatrixFlat$Wert1 != corrMatrixFlat$Wert2 & abs(corrMatrixFlat$Korrelation) > 0.4,]

pdf(file = "Corr_Flat_Alternativ.pdf", height=9, width=5)
grid.table(corrMatrixFlat)
dev.off()

                   Geschlecht    B3_Fragen     B1_Präf    B1_Fragen
Geschlecht        1.000000000 -0.008349681  0.03999072  0.002674323
B3_Fragen        -0.008349681  1.000000000 -0.03202195  0.375385293
B1_Präf           0.039990723 -0.032021950  1.00000000 -0.070810334
B1_Fragen         0.002674323  0.375385293 -0.07081033  1.000000000
B2_Kennen         0.119582028 -0.032003650  0.30965821 -0.094374935
B2_Besitzen       0.117930307  0.014568222  0.42876771  0.003808865
B2_Kaufen         0.190862284  0.063678234  0.16470551  0.031423631
B2_Ästhetik      -0.017447753  0.069130138  0.22099681 -0.046440948
B2_Alltagstaugl.  0.058371919  0.114860854  0.16184126  0.005607784
B2_Kosten         0.008279476 -0.060807788  0.03304134  0.013196219
B3_Präf           0.104755271 -0.044209965  0.68902198 -0.034591515
Umwelt.FB         0.250340317  0.105664573  0.19366051  0.038189163
                     B2_Kennen B2_Besitzen  B2_Kaufen B2_Ästhetik
Geschlecht        0.1195820277 0.117930307 0.19086

png 
  2

png 
  2

png 
  2

In [37]:
#Erstelle subset für Korrelations-Berechnung
dataType = rawData[rawData$Typ == "Plastik", c(3,5:7,9:13,15,17)]
#dataType$VB = as.integer(as.factor(dataType$VB))
dataType$Geschlecht = as.integer(as.factor(dataType$Geschlecht))

# korrelationsn berechnen
corrMatrix = cor(dataType)

print(corrMatrix)

#CorrMatrix erstellen
cor3 = corrplot(corrMatrix, method="circle", type = "upper", sig.level = .05)
cor4 = corrplot(corrMatrix, method = "number", type = "upper", sig.level = .05, number.cex = 8/ncol(corrMatrix))

pdf(file = "cor3.pdf")
corrplot(corrMatrix, method="circle", type = "upper", sig.level = .05)
dev.off()

pdf(file = "cor4.pdf")
corrplot(corrMatrix, method = "number", type = "upper", sig.level = .05, number.cex = 8/ncol(corrMatrix))
dev.off()


#Erstelle Tabelle nur mit hoch korrelierten features
corrMatrixFlat = as.data.frame(as.table(corrMatrix), stringsAsFactors = FALSE)
colnames(corrMatrixFlat) = c("Wert1", "Wert2", "Korrelation")
corrMatrixFlat = corrMatrixFlat[corrMatrixFlat$Wert1 != corrMatrixFlat$Wert2 & abs(corrMatrixFlat$Korrelation) > 0.4,]

pdf(file = "Corr_Flat_Plastik.pdf", height=9, width=5)
grid.table(corrMatrixFlat)
dev.off()

                   Geschlecht    B3_Fragen     B1_Präf   B1_Fragen  B2_Besitzen
Geschlecht        1.000000000 -0.153256466 -0.14283123 -0.09292781 -0.006985496
B3_Fragen        -0.153256466  1.000000000  0.03515636  0.24987865  0.009635151
B1_Präf          -0.142831235  0.035156362  1.00000000  0.06636497  0.458364961
B1_Fragen        -0.092927814  0.249878651  0.06636497  1.00000000  0.054917823
B2_Besitzen      -0.006985496  0.009635151  0.45836496  0.05491782  1.000000000
B2_Kaufen        -0.096225045  0.024131635  0.52776737  0.03219114  0.689656308
B2_Ästhetik      -0.034531457  0.074678385  0.27587274  0.12365088  0.216373750
B2_Alltagstaugl. -0.009743243  0.009341849  0.35815737  0.04360823  0.392646095
B2_Kosten        -0.022271185  0.091354855  0.16926137  0.14118430  0.032801217
B3_Präf          -0.177148815  0.092105349  0.65579179  0.11403712  0.367023030
Umwelt.FB         0.250340317 -0.012568473 -0.20849127 -0.05353072 -0.159320439
                   B2_Kaufen B2_Ästhetik

png 
  2

png 
  2

png 
  2